In [264]:
from itertools import chain
from collections import deque
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import sys
import Graph as G

In [160]:
movie_actors = pd.read_csv ("data/movie-actors.txt", sep = '|')
movie_actors.columns = ["movie", "name"]
movie_actors.head(1025)

,movie,name
0,1,3
1,1,5
2,1,4
3,2,10
4,2,7
...,...,...
1020,316,1064
1021,316,351
1022,317,2
1023,317,360


In [163]:
actors = pd.read_csv("data/actors.txt", sep = "|", header = None)
actors.columns = ["id", "name"]
actors.set_index("id", inplace = True)
actors.head()

,name
id,
2,Tim Allen
3,Tom Hanks
4,Don Rickles
5,Jim Varney
7,Jonathan Hyde


In [164]:
movies = pd.read_csv("data/movies.txt", sep = "|", header = None)
movies.columns = ["id", "name"]
movies.set_index("id", inplace = True)
movies.head(1009)

,name
id,
1,Toy Story (1995)
2,Jumanji (1995)
3,Grumpier Old Men (1995)
4,Waiting to Exhale (1995)
5,Father of the Bride Part II (1995)
...,...
1021,Angels in the Outfield (1994)
1022,Cinderella (1950)
1023,Winnie the Pooh and the Blustery Day (1968)


In [165]:
movie_list = []
actor_list = []

for row in range(len(movie_actors['movie'])):
    m_value = movies['name'][movie_actors['movie'][row]]
    a_value = actors['name'][movie_actors['name'][row]]
    movie_list.append(m_value)
    actor_list.append(a_value)
    
df = pd.DataFrame(list(zip(movie_list, actor_list)), columns = ["Title", "Name"])
df.head()

,Title,Name
0,Toy Story (1995),Tom Hanks
1,Toy Story (1995),Jim Varney
2,Toy Story (1995),Don Rickles
3,Jumanji (1995),Kirsten Dunst
4,Jumanji (1995),Jonathan Hyde


In [179]:
g = G.Graph()
unique_actor = df["Name"].unique()
for actor in unique_actor:
    g.add_vertex(actor)

In [167]:
unique_movie = df["Title"].unique()
print(unique_movie)

['Toy Story (1995)' 'Jumanji (1995)' 'Grumpier Old Men (1995)' ...
 'Chasing Liberty (2004)' "My Baby's Daddy (2004)" 'Distant (Uzak) (2002)']


In [180]:
current_count = 0
names = []

for i in range(len(df)):
    current_title = unique_movie[current_count]
    if df['Title'][i] != current_title:
        if len(names) > 1:
            for x in range(len(names)):
                for j in range(1+x, len(names)):
                    g.get_vertex(names[x]).add_neighbor(names[j], i)
        
        current_count = current_count + 1            
        names = []
                       
    names.append(df['Name'][i])

In [451]:
def shortest(graph, start, goal = "Kevin Bacon"):
    if graph.get_vertex(start) == None:
        print("Actor not found")
        return
    ng = nx.Graph()
    explored = []
    queue = [[start]]
    weights = []
    
    if start == goal:
        print("Same Node")
        return
    while queue:
        path = queue.pop(0)
        node = path[-1]
        
        if node not in explored:
            neighbours = graph.get_vertex(node).connected_to
            for neighbour in neighbours:
                new_path = list(path)
                new_path.append(neighbour)
                queue.append(new_path)
                
                if neighbour == goal:
                    ng.add_nodes_from(new_path)
                    print(start +"'s number is "+str(len(new_path)))
                    for p in range(1, len(new_path)):
                        weights.append(graph.get_vertex(new_path[p-1]).get_weight(new_path[p]))
                    for x in range(len(new_path)):
                        if x != len(new_path) - 1:
                            ng.add_edge(new_path[x], new_path[x+1], v = df['Title'][weights[x]])
                            print(new_path[x] + " appeared in " + 
                                  df['Title'][weights[x]]+ " with "  + new_path[x+1])
                    
                    pos = nx.spring_layout(ng, 6)
                    plt.figure() 
                    nx.draw(ng, pos, edge_color='black',width=2,linewidths=2, 
                            with_labels=True, node_size=1000,node_color='lightblue',
                            alpha=0.8, labels={node:node for node in ng.nodes()})
                    edge_labels = nx.get_edge_attributes(ng,'v')
                    nx.draw_networkx_edge_labels(ng, pos, edge_labels = edge_labels, 
                                                font_color='green', font_size=8)
                    plt.axis('off')
                    axis = plt.gca()
                    axis.set_xlim([1.2*x for x in axis.get_xlim()])
                    axis.set_ylim([1.2*y for y in axis.get_ylim()])
                    plt.show()
                    return
            explored.append(node)
    print("No connecting path exists.")

In [452]:
def distance(graph, start = "Kevin Bacon"):    
    explored = []
    queue = [[start]]
    maxDis = 0
    avrDis = 0
    count = 0
    maxDisAct = ""
    
    while queue:
        path = queue.pop(0)
        node = path[-1]
        
        if node not in explored:
            neighbours = graph.get_vertex(node).connected_to
            for neighbour in neighbours:
                new_path = list(path)
                new_path.append(neighbour)
                queue.append(new_path)
                distance = len(new_path)
                avrDis = avrDis + distance
                count = count + 1
                if distance > maxDis:
                    maxDis = distance
                    maxDisAct = new_path[distance-1]
            explored.append(node)
    avrDis = avrDis/count
    return maxDis, maxDisAct, avrDis

In [453]:
def getInput():
    print("\nTo quit the program, type return in answer to a question.")
    name = input("Enter the name of the actor: ")
    if name is "\n":
        quit()
    maxDis, maxDisAct, avrDis = distance(g)
    print("The Actor with the farthest distance from Kevin Bacon is " + maxDisAct + " at "+ str(maxDis))
    print("The average distance from Kevin Bacon is "+ str(avrDis)+"\n")
    shortest(g, name)


In [455]:
getInput()


To quit the program, type return in answer to a question.
Enter the name of the actor: 1
The Actor with the farthest distance from Kevin Bacon is Raymond Massey at 10
The average distance from Kevin Bacon is 5.08051480277607

Actor not found
